# Capstone Project : The Battle of Neighborhoods

One of my friends lives in Kuala Lumpur, Malayasia. He works as a Builder. A client of his asked to locate the areas best suited for constructing a shopping mall in the city of Kuala Lumpur. So this project is aimed at identifying such places. First, we will build a dataframe containing all the neighborhoods and their geographic coordinates. Then, we wil fetch the data of venues using the Foursqaure API. After this, we will select the venues of shopping mall and cluster them. Finally, we will select the best cluster to construct the shopping mall

### Importing Necessary Libraries

In [13]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import folium
from geopy.geocoders import Nominatim 
!pip install geocoder

import requests
import json
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [14]:
import geocoder

### Biulding dataframe containing neighborhood geographical coordinates

In [17]:
df = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')

In [18]:
neighborList = []

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborList.append(row.text)

In [20]:
data = pd.DataFrame({"Neighborhood": neighborList})
data.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [25]:
def get_latlong(neighborhood):
    coordinates = None
    
    while(coordinates is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        coordinates = g.latlng
    return coordinates

In [26]:
neighbor_coor = [ get_latlong(neighborhood) for neighborhood in data["Neighborhood"].tolist() ]

In [27]:
neighbor_coor[0:5]

[[3.0576900000000364, 101.74388000000005],
 [3.1484988508598852, 101.69672774991264],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005]]

In [28]:
coor_df = pd.DataFrame(neighbor_coor, columns = ['Latitude', 'Longitude'])

In [29]:
data['Latitude'] = coor_df['Latitude']
data['Longitude'] = coor_df['Longitude']

In [30]:
data.head()

,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [32]:
add = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(add)
lat = location.latitude
long = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysia {}, {}.'.format(lat, long))

The geograpical coordinate of Kuala Lumpur, Malaysia 3.1516964, 101.6942371.


### Visualising data on Kuala Lumpur map 

In [33]:
kl_map = folium.Map(location = [lat, long], zoom_start = 11)

# add markers to map
for lat, lng, neighborhood in zip( data['Latitude'], data['Longitude'], data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(kl_map)  
    
kl_map

### Providing Foursqaure Credentials

In [35]:
client_id = '0NK31O1O5Q3NTQXZIIUWNUXMP2VFEU1ZHC5BNIVTIZR5R2ZR'
client_secret = 'IQ0TUL0M2VCWYWGKGKO2U43Y5VQXK0YCGZ5HBPBJJKPMTZP5'
version = '20180605'
radius = 2000
limit = 100

### Fetching Venue data using Foursqaure API

In [ ]:
venue_info = []

for lat, lng, neighborhood in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    
    # create the API request URL
    html = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        client_id,
        client_secret,
        version,
        lat,
        lng,
        radius, 
        limit)
    
    result = requests.get(html).json()["response"]['groups'][0]['items']
    
    for venue in result:
        venue_info.append((
            neighborhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

### Merging venue and neighborhood data

In [38]:
df_venue = pd.DataFrame(venue_info)
df_venue.columns = ['Neighborhood', 'Latitude', 'Longitude', 'Venue Name', 'Venue Lat', 'Venue Long', 'Venue Category']
df_venue.head()

,Neighborhood,Latitude,Longitude,Venue Name,Venue Lat,Venue Long,Venue Category
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
4,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant


In [39]:
df_venue.shape

(7087, 7)

In [41]:
df_venue['Venue Category'].unique()

array(['Supplement Shop', 'Food & Drink Shop', 'Noodle House',
       'Chinese Restaurant', 'Seafood Restaurant', 'Breakfast Spot',
       'Restaurant', 'Food Court', 'Vegetarian / Vegan Restaurant',
       'Dessert Shop', 'Park', 'Bubble Tea Shop', 'Coffee Shop',
       'Asian Restaurant', 'Indian Restaurant', 'Other Great Outdoors',
       'Pet Store', 'Food Truck', 'Spa', 'Snack Place',
       'Dim Sum Restaurant', 'Convenience Store',
       'Chinese Breakfast Place', 'Japanese Restaurant', 'Outlet Store',
       'Bakery', 'Café', 'Farmers Market', 'Gym / Fitness Center',
       'Malay Restaurant', 'Athletics & Sports', 'Steakhouse',
       'Cantonese Restaurant', 'Fast Food Restaurant', 'Hakka Restaurant',
       'Badminton Court', 'Mamak Restaurant', 'Middle Eastern Restaurant',
       'Burger Joint', 'Winery', 'College Bookstore', 'Grocery Store',
       'Diner', 'Vietnamese Restaurant', 'Gift Shop', 'Soup Place',
       'Salon / Barbershop', 'Flea Market', 'Hostel',
       'Sou

In [48]:
dummy = pd.get_dummies(df_venue[['Venue Category']], prefix="", prefix_sep="")

dummy['Neighborhood'] = df_venue['Neighborhood'] 

fixed_columns = [dummy.columns[0]] + list(dummy.columns[:-1])
dummy = dummy[fixed_columns]
dummy.head()

,Accessories Store,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,

In [50]:
first = dummy.pop('Neighborhood')

In [52]:
dummy.insert(0, 'Neighborhood', first)
dummy.head()

,Neighborhood,Accessories Store,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,

In [55]:
group_dummy = dummy.groupby(['Neighborhood']).mean().reset_index()
group_dummy.head()

,Neighborhood,Accessories Store,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,

In [56]:
len(group_dummy[group_dummy["Shopping Mall"] > 0])

38

### Creating dataframe containing only shopping malls

In [57]:
df_mall = group_dummy[['Neighborhood','Shopping Mall']]

In [58]:
df_mall.head()

,Neighborhood,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.010000
2,Bandar Menjalara,0.010000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.010417


### Applying K Means Clustering

In [59]:
train = df_mall.drop('Neighborhood', axis = 1)

kmeans = KMeans(n_clusters = 4, random_state = 2)
kmeans.fit(train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2, tol=0.0001, verbose=0)

In [63]:
kmeans.labels_[0:20]

array([0, 3, 3, 0, 3, 0, 1, 1, 2, 0, 0, 2, 2, 0, 0, 2, 3, 2, 0, 2],
      dtype=int32)

In [69]:
df_mall1 = df_mall.copy()
df_mall1['Cluster Label'] = kmeans.labels_
df_mall1.rename(columns = {"Neighborhoods": "Neighborhood"}, inplace=True)
df_mall1.head(10)

,Neighborhood,Shopping Mall,Cluster Label
0,Alam Damai,0.000000,0
1,"Ampang, Kuala Lumpur",0.010000,3
2,Bandar Menjalara,0.010000,3
3,Bandar Sri Permaisuri,0.000000,0
4,Bandar Tasik Selatan,0.010417,3
5,Bandar Tun Razak,0.000000,0
6,Bangsar,0.050000,1
7,Bangsar Park,0.050000,1
8,Bangsar South,0.020000,2
9,Batu 11 Cheras,0.000000,0


In [70]:
df_mall1 = df_mall1.join(data.set_index("Neighborhood"), on="Neighborhood")
df_mall1.head()

,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
0,Alam Damai,0.000000,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.010000,3,3.148499,101.696728
2,Bandar Menjalara,0.010000,3,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.010417,3,3.072750,101.714610


In [83]:
df_mall1.sort_values(["Cluster Label"], inplace=True)
df_mall1.reset_index(inplace =True)
df_mall1.head()

,level_0,index,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
0,0,0,Alam Damai,0.0,0,3.05769,101.74388
1,32,33,Kampung Padang Balang,0.0,0,3.20943,101.69318
2,31,34,Kepong,0.0,0,3.21750,101.63763
3,30,36,Kuchai Lama,0.0,0,3.09074,101.67733
4,29,38,Maluri,0.0,0,3.14789,101.69405


### Analysing Cluster 0

In [85]:
df_mall1.loc[df_mall1['Cluster Label'] == 0]

,level_0,index,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
0,0,0,Alam Damai,0.0,0,3.057690,101.743880
1,32,33,Kampung Padang Balang,0.0,0,3.209430,101.693180
2,31,34,Kepong,0.0,0,3.217500,101.637630
3,30,36,Kuchai Lama,0.0,0,3.090740,101.677330
4,29,38,Maluri,0.0,0,3.147890,101.694050
5,28,40,Miharja,0.0,0,3.147890,101.694050
6,27,42,Pantai Dalam,0.0,0,3.094760,101.667470
7,26,45,Salak South,0.0,0,3.081540,101.696890
8,25,47,Semarak,0.0,0,3.179943,101.721449
9,24,49,Setapak,0.0,0,3.188160,101.704150


### Analysing Cluster 1

In [86]:
df_mall1.loc[df_mall1['Cluster Label'] == 1]

,level_0,index,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
33,37,7,Bangsar Park,0.05,1,3.129200,101.678440
34,38,6,Bangsar,0.05,1,3.129200,101.678440
35,36,27,"Jalan Cochrane, Kuala Lumpur",0.04,1,3.134630,101.721690
36,35,37,Lembah Pantai,0.04,1,3.121189,101.663889
37,34,67,Taman U-Thant,0.04,1,3.157700,101.724520
38,33,43,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070


### Analysing Cluster 2

In [87]:
df_mall1.loc[df_mall1['Cluster Label'] == 2]

,level_0,index,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
39,55,19,Chow Kit,0.02,2,3.163590,101.698110
40,54,46,Segambut,0.02,2,3.186390,101.668100
41,52,52,Sri Hartamas,0.02,2,3.162200,101.650360
42,51,51,Shamelin,0.02,2,3.124570,101.735970
43,50,12,Bukit Bintang,0.02,2,3.147770,101.708550
44,49,30,KL Eco City,0.02,2,3.117130,101.673840
45,48,66,Taman Tun Dr Ismail,0.03,2,3.152830,101.622710
46,47,20,Damansara Heights,0.02,2,3.147970,101.667950
47,53,23,Dang Wangi,0.02,2,3.157825,101.697280
48,45,8,Bangsar South,0.02,2,3.111020,101.662830


### Analysing Cluster 3

In [102]:
df_mall1.loc[df_mall1['Cluster Label'] == 3]

,level_0,index,Neighborhood,Shopping Mall,Cluster Label,Latitude,Longitude
56,68,2,Bandar Menjalara,0.010000,3,3.190350,101.625450
57,67,57,Taman Desa,0.010000,3,3.102970,101.684710
58,66,4,Bandar Tasik Selatan,0.010417,3,3.072750,101.714610
59,65,1,"Ampang, Kuala Lumpur",0.010000,3,3.148499,101.696728
60,64,25,"Federal Hill, Kuala Lumpur",0.010000,3,3.136370,101.685640
61,63,54,Sungai Besi,0.010000,3,3.049970,101.706030
62,59,32,Kampung Datuk Keramat,0.010000,3,3.166400,101.730460
63,61,16,Bukit Petaling,0.010000,3,3.129290,101.698960
64,60,69,Titiwangsa,0.010000,3,3.180730,101.703210
65,58,22,"Damansara, Kuala Lumpur",0.010000,3,3.138766,101.684015


In [101]:
cluster_map = folium.Map(location=[lat, long], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(4)
y = [i + x + (i*x)**2 for i in range(4)]
rainbow = ['red','blue','green','yellow']

for lat, lon, n, cluster in zip(df_mall1['Latitude'], df_mall1['Longitude'], df_mall1['Neighborhood'], df_mall1['Cluster Label']):
    label = folium.Popup(str(n) + ' - Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(cluster_map)
       
cluster_map

### Conclusion :

From the above analysis, it seems that neighborhoods in Cluster 1 and 2 are densely populated with shopping malls and therefore, the owners have to face an intense competition. On the other hand, Cluster 0 has a very few number of shopping malls. Therefore, neighborhoods in Cluster 0 are suitable for the construction of a new shopping mall because they have a little or no competition and there are high chances that the shopping mall will produce more profit for the client. However, Cluster 3 can also be taken into consideration as there are only few shopping malls there. Lastly, the client should be suggested to avoid cluster 1 and 2 as they are densely populated with shopping malls. 